### In this notebook I will be preprocessing the data and building an LDA model to model the news articles into their respective groups.

In [1]:
import nltk; 
import re
import numpy as np
import pandas as pd
import itertools
import spacy
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings('ignore')
stop_words = stopwords.words('english')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\3aksh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\3aksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Helper Functions

In [2]:
#Function to compute rand index
def compute_rand_index(documents_list,cluster_labels_LDA,cluster_labels_D2V):
        X_dict = dict(zip(documents_list, cluster_labels_LDA)) 
        Y_dict = dict(zip(documents_list, cluster_labels_D2V))
        r = 2
        a = 0
        b = 0
        total = 0
        for i,j in list(itertools.combinations(documents_list, r)):
            if (X_dict.get(i) == X_dict.get(j) and Y_dict.get(i) == Y_dict.get(j)) :
                a += 1
            elif (X_dict.get(i) != X_dict.get(j) and Y_dict.get(i) != Y_dict.get(j)):
                b += 1
            total += 1
        return float(a+b) / total

# Function to prepare data    
def prepare_data(data):
    #Seperate data into individual lists from the dictionary
    documents = data["data"]
    doc_id = np.arange(len(documents))
    #Seperate document labels into individual lists from the dictionary
    target = data["target"]
    #Create a dataframe
    df = pd.DataFrame({'doc_id': doc_id,'documents':documents,'class_no':target})
    return df

# Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Function to lemmatize the data
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
    
# Create Dictionary
def create_dictionary_LDA(data):
    # Create Dictionary
    id2word = corpora.Dictionary(data)
    # Create Corpus
    texts = data
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    return corpus, id2word


#Function to get RI scores of LDA clustering
def get_RandIndex_LDA(num_of_topics):
    list_of_RI = []
    
    
    for t in num_of_topics:
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=t, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=1,
                                           alpha='symmetric',
                                           per_word_topics=True)
        
        # Assigns the topics to the documents in corpus

        doc_topic = lda_model.get_document_topics(corpus, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)
        
        #seperating topic distribution
        
        topic_distribution = []
        for j in zip(doc_topic,doc_id):
            topic_distribution.append(j[0])#seperating topic distribution
            
        #print(len(topic_distribution))
        #to get list of max probabilities for each document
        
        list_of_max_prob = []
        for list_ in topic_distribution:
            list_of_max_prob.append(max(list_, key=lambda x:x[1]))#to compare using second value in a tuple that is the probability distribution
        #print(len(list_of_max_prob))
        #to get the clusters to which each document belongs to
        cluster_lda = []
        for i in list_of_max_prob:
            cluster_lda.append(i[0])#Final list of clusters based on max probability
            
        list_of_RI.append(compute_rand_index(docs,cluster_lda,ground_truth))
            
    return list_of_RI, cluster_lda, lda_model
    
#Save model predictions
def save_clusters(clusters, data):
    data['LDA_clusters'] = clusters
    data.to_csv("model_predictions.csv")

### Import Dataset

In [3]:
newsgroups_data = fetch_20newsgroups()

### Preparing the data

In [4]:
newsgroups_data_df = prepare_data(newsgroups_data)

In [5]:
newsgroups_data_df.head(10)

,doc_id,documents,class_no
0,0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
5,5,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,16
6,6,From: bmdelane@quads.uchicago.edu (brian manni...,13
7,7,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,3
8,8,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,2
9,9,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,4


In [6]:
docs = newsgroups_data_df.documents.tolist()
doc_id = newsgroups_data_df.doc_id.values.tolist()
ground_truth = newsgroups_data_df.class_no.values.tolist()

### Cleaning the data

In [7]:
# Convert to list
data = newsgroups_data_df.documents.values.tolist()

# deacc=True removes punctuations
data_words = list(sent_to_words(data))

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Creating Bigrams
bigram = gensim.models.Phrases(data_words_nostops) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## LDA topic models

### Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Get Dictionary
corpus, id2word = create_dictionary_LDA(data_lemmatized)

## Building the LDA Model and getting the Rand Index values

In [9]:
number_of_clusters = [20]
list_of_RI, cluster_lda, lda_model = get_RandIndex_LDA(number_of_clusters)

In [10]:
print("\n The Rand Index score for the LDA clusters when compared with the true labels is : " + str(list_of_RI[0]))


 The Rand Index score for the LDA clusters when compared with the true labels is : 0.8721040983369996


### Save the predictions

In [11]:
newsgroups_data_df['LDA_clusters'] = cluster_lda
newsgroups_data_df.to_csv("model_predictions.csv")